In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm


In [2]:
!git clone https://github.com/keshavshrikant/MachineLearning.git

fatal: destination path 'MachineLearning' already exists and is not an empty directory.


In [ ]:
!cd MachineLearning && git checkout kaggle-gemma

In [3]:
import json
transformations = []
with open("MachineLearning/gpt_generated_tranformations.jsonl", "r") as fin:
    for line in fin:
        instruction_info = json.loads(line)
#         print(instruction_info)
        transformations.append(instruction_info) 

trans_df = pd.DataFrame.from_records(transformations)
trans_df.head()

,instruction,actual_text,rewritten_text
0,"Start with a hypothetical outcome, exploring v...","Hi Delta Airline,\n\nI am Ao Ni, I send this e...",In a parallel universe where my flight from La...
1,Highlight a background detail that was previou...,Swimming in the ocean can be an exhilarating e...,The sound of crashing waves against the shore ...
2,Develop a list of potential spin-off stories f...,Below is a recursive Python function fib(n) th...,"Within the world of the Fibonacci sequence, th..."
3,"Write a letter from one character to another, ...",* Planned fundraising efforts for the next wee...,"Dear Sarah, I wanted to update you on the prog..."
4,Develop a step-by-step guide on how to cultiva...,"Hi Landlord, I wanted to send this message to ...",To cultivate a thriving indoor garden in a sma...


In [4]:
from kaggle_secrets import UserSecretsClient

os.environ["KAGGLE_USERNAME"] = UserSecretsClient().get_secret('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = UserSecretsClient().get_secret('KAGGLE_KEY')

In [5]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [6]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [7]:
import keras
import keras_nlp

2024-03-14 11:32:30.357851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 11:32:30.357901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 11:32:30.359337: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [9]:
prompt_recovery_template = """Provided below is a creative writing paragraph and an interesting manner in which it has been rewritten. Your job is to figure out the insturction which would have been followed to rewrite the original paragraph in the interesting manner. Identify the style, tone, structure, point of view, theme, genre in which the original pragraph has been rewritten and then reconstruct the instruction.

Here is an example:
Original Text:
The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

Rewritten Text:
Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

Instruction:
Convert this into a sea shanty

Now do it for this one!

Original Text:
{para}

Rewritten Text:
{rewritten}

Instruction:
{instruction}
"""


In [10]:
train_df = []
    
for index, row in trans_df.iterrows():

    prompt = prompt_recovery_template.format(
        para=row['actual_text'],
        rewritten=row['rewritten_text'],
        instruction=row['instruction'],
    )

    if len(prompt) < 3000:
        
        train_df.append(prompt)

In [11]:
len(train_df)

506

In [17]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.preprocessor.sequence_length = 3000

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
    beta_1=0.9,
    beta_2=0.999
    )

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

ValueError: lora is already enabled. This can only be done once per layer.

In [13]:
gemma_lm.fit(train_df, epochs=2, batch_size=1)

Epoch 1/2
506/506 ━━━━━━━━━━━━━━━━━━━━ 632s 1s/step - loss: 1.2524 - sparse_categorical_accuracy: 0.5858
Epoch 2/2
506/506 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - loss: 0.5488 - sparse_categorical_accuracy: 0.8112


In [15]:
test_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

prompt = prompt_recovery_template.format(
    para=test_df['original_text'].iloc[0],
    rewritten=test_df['rewritten_text'].iloc[0],
    instruction=""
)

response = gemma_lm.generate(prompt,max_length=3000)

In [16]:
response = response.replace(prompt,'')
response

"Copy the text below:\n\nThe text above is a secret message. Can you decrypt it?\n\nThe answer to the above question is:\nThe text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be\n</code>\n\nThis is the original text.\n\nHere is your shanty:\n\nThe text is rewritten, the LLM has spun,\n\nWith prompts so clever,\n\nthey've been outrun.\n\nThe goal is to find, the prompt so bright,\n\nTo crack the code, and shine the light.\n\nOh, this is a code competition, my dear,\n\nWith text and prompts,\n\nwe'll compete.\n\nTwo thousand texts, a challenge grand,\n\nTo guess the prompts,\n\nhand over hand.(Verse 2) The original text,\n\na treasure l

In [ ]:
prompt

In [ ]:
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)